In [1]:
# import needed packages
import pandas as pd
import requests
from bs4 import BeautifulSoup
import math
import re
import time

In [2]:
# get id function
def get_id(term):
    id_list = []
    API_KEY = "59cbd6ff6b3ce5d506f0eeb69e4e09362409"
    url_search = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=gds&api_key={API_KEY}&term={term}&retmax=200"
    info = BeautifulSoup(requests.get(url_search).text, 'lxml')
    
    term_count = int(info.find('count').text)
    id_list += [single_term.text for single_term in info.find_all('id')]
    
    for page in range(1, math.ceil(term_count/200)):
        page_url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=gds&api_key={API_KEY}&term={term}&retmax=200&retstart={page*200}"
        page_info = BeautifulSoup(requests.get(page_url).text, 'lxml')
        id_list += [single_term.text for single_term in page_info.find_all('id')]
    
    return id_list, term_count


In [3]:
term = "(PBMC)+AND+(single-cell)"
id_list, term_count = get_id(term=term)

/home/ouyangkang/software/anaconda3/envs/ML/lib/python3.9/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [4]:
# check result
print(len(set(id_list)) == term_count)

True


In [6]:
# reserve result
with open('/home/ouyangkang/Learning/Python/other_thing/id_result.txt', 'w') as f:
    f.writelines("\n".join(id_list))

In [2]:
with open('/home/ouyangkang/Learning/Python/other_thing/id_result.txt', 'r') as f:
    id_list = [i.strip('\n') for i in f.readlines()]

In [6]:
# define a function to get detailed info
def id_detailed(id_list): # the max capacity of id list is 20
    API_KEY = "59cbd6ff6b3ce5d506f0eeb69e4e09362409"
    return_list = []
    
    for page in range(math.ceil(len(id_list)/20)):
        id_search = ",".join(id_list[page*20:(page+1)*20])
        id_url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=gds&api_key={API_KEY}&id={id_search}&rettype=summary"
        id_info = BeautifulSoup(requests.get(id_url).text, 'lxml')

        # Title
        title = [i.text for i in id_info.find_all('item', {'name': 'title'})]
        # Summary
        summary = [i.text for i in id_info.find_all('item', {'name': 'summary'})]
        # GSE
        gse = [i.text for i in id_info.find_all('item', {'name': 'GSE'})]
        # Taxon
        taxon = [i.text for i in id_info.find_all('item', {'name': 'taxon'})]
        # sequencing type
        seq_type = [i.text for i in id_info.find_all('item', {'name': 'gdsType'})]
        # Publish date
        p_date = [i.text for i in id_info.find_all('item', {'name': 'PDAT'})]
        try:
            for num in range(20):
                return_list.append([title[num], summary[num], "GSE"+gse[num], taxon[num], seq_type[num], p_date[num]])
        # print(f"the {page*20}-{(page+1)*20} ids have finished")
        except:
            print(id_search)

    return return_list

In [7]:
result_info = id_detailed(id_list=id_list)

/home/ouyangkang/software/anaconda3/envs/ML/lib/python3.9/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


302041673,302041672,302041671,302041670,302041669,301826282,301361942,301361941,301361940,301361939


In [11]:
# check length of each element
for i in range(len(result_info)):
    if len(result_info[i]) != 6:
        print(result_info[i])

In [14]:
# filter non-homo sapiens data and store result
no_filtering_result = pd.DataFrame(result_info, columns=['Title', 'Summary', 'GSE', 'Taxon', 'Sequencing_type', 'Publish_date'])
filtering_result = no_filtering_result[no_filtering_result["Taxon"] == "Homo sapiens"]
filtering_result.to_csv('/mnt/c/Users/ouyangkang/Desktop/GEO_filtering_result_11_3.csv', index=None)

In [20]:
filtering_result.head()

,Title,Summary,GSE,Taxon,Sequencing_type,Publish_date
0,Pathophysiology of androgen-associated endothe...,Abundant evidence suggests that women with pol...,GSE246456,Homo sapiens,Expression profiling by high throughput sequen...,2023/11/01
1,Immune landscape of PBMC based on single-cell ...,Although an increase in studies has revealed t...,GSE233315,Homo sapiens,Expression profiling by high throughput sequen...,2023/10/31
2,Gene expression profile at single cell level o...,We used single cell RNA sequencing (scRNAseq) ...,GSE229626,Homo sapiens,Expression profiling by high throughput sequen...,2023/10/31
3,Gene expression profiles of peripheral blood m...,An autoimmune B cell origin for childhood idio...,GSE233276,Homo sapiens,Expression profiling by high throughput sequen...,2023/10/29
4,Gene expression profiles of peripheral blood m...,An autoimmune B cell origin for childhood idio...,GSE233275,Homo sapiens,Expression profiling by high throughput sequen...,2023/10/29


In [19]:
len(set(id_list))

7150

In [3]:
API_KEY = "59cbd6ff6b3ce5d506f0eeb69e4e09362409"
id_search = ",".join(id_list[:20])
id_url = id_url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=gds&api_key={API_KEY}&id={id_search}&rettype=summary"
id_info = BeautifulSoup(requests.get(id_url).text, 'lxml')

/home/ouyangkang/software/anaconda3/envs/ML/lib/python3.9/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [4]:
id_info

<?xml version="1.0" encoding="UTF-8" ?><!DOCTYPE eSummaryResult PUBLIC "-//NLM//DTD esummary v1 20041029//EN" "https://eutils.ncbi.nlm.nih.gov/eutils/dtd/20041029/esummary-v1.dtd">
<html><body><esummaryresult>
<docsum>
<id>200246456</id>
<item name="Accession" type="String">GSE246456</item>
<item name="GDS" type="String"></item>
<item name="title" type="String">Pathophysiology of androgen-associated endothelial cell dysfunction in phenotype A polycystic ovarian syndrome revealed by iPSCs modeling</item>
<item name="summary" type="String">Abundant evidence suggests that women with polycystic ovary syndrome (PCOS) have increased metabolic aberrations and cardiovascular risks and present with signs of endothelial cell (EC) dysfunction. However, whether and how androgen is involved in the pathogenesis of EC dysfunction in PCOS remains unclear. Induced pluripotent stem cells (iPSCs) were established in two PCOS patients and two control participants and further differentiated into ECs via mo

In [10]:
[i.text for i in id_info.find_all('item', {'name': 'Accession'})]

['GSE246456',
 'GSM7869174',
 'GSM7869175',
 'GSM7869176',
 'GSM7869173',
 'GSE233315',
 'GSM7422420',
 'GSM7422419',
 'GSE229626',
 'GSM7169474',
 'GSM7169477',
 'GSM7169480',
 'GSM7169483',
 'GSM7169486',
 'GSM7169489',
 'GSM7169472',
 'GSM7169478',
 'GSM7169481',
 'GSM7169475',
 'GSM7169487',
 'GSM7169484',
 'GSM7169473',
 'GSM7169476',
 'GSM7169479',
 'GSM7169482',
 'GSM7169488',
 'GSM7169485',
 'GSE233276',
 'GSM7421517',
 'GSM7421514',
 'GSM7421512',
 'GSM7421518',
 'GSM7421515',
 'GSM7421513',
 'GSM7421516',
 'GSM7421519',
 'GSE233275',
 'GSM7421507',
 'GSM7421510',
 'GSM7421505',
 'GSM7421508',
 'GSM7421511',
 'GSM7421506',
 'GSM7421509',
 'GSM7421504',
 'GSE232117',
 'GSM7315514',
 'GSM7315477',
 'GSM7315494',
 'GSM7315474',
 'GSM7315471',
 'GSM7315517',
 'GSM7315491',
 'GSM7315497',
 'GSM7315503',
 'GSM7315457',
 'GSM7315523',
 'GSM7315483',
 'GSM7315520',
 'GSM7315500',
 'GSM7315480',
 'GSM7315463',
 'GSM7315460',
 'GSM7315526',
 'GSM7315506',
 'GSM7315486',
 'GSM7315512',
 